# Models

Bu bölümde bir model oluşturmaya ve kullanmaya daha yakından bakacağız. Herhangi bir modeli bir kontrol noktasından örneklemek istediğinizde kullanışlı olan AutoModel sınıfını kullanacağız.

AutoModel sınıfı ve tüm akrabaları aslında kütüphanede bulunan çok çeşitli modeller üzerinde basit sarmalayıcılardır. Kontrol noktanız için uygun model mimarisini otomatik olarak tahmin edebildiği ve ardından bu mimariye sahip bir modeli örneklediği için akıllıca bir sarmalayıcıdır.

Ancak, kullanmak istediğiniz modelin türünü biliyorsanız, doğrudan mimarisini tanımlayan sınıfı kullanabilirsiniz. Bunun bir BERT modeli ile nasıl çalıştığına bir göz atalım.

## Creating a Transformer

Bir BERT modelini başlatmak için yapmamız gereken ilk şey bir yapılandırma nesnesi yüklemektir:

In [1]:
from transformers import BertConfig, BertModel

config = BertConfig()
model = BertModel(config)

Yapılandırma(configuration), modeli oluşturmak için kullanılan birçok öznitelik içerir:

In [2]:
print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.42.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



Bu niteliklerin tümünün ne işe yaradığını henüz görmemiş olsanız da, bazılarını tanımanız gerekir: **hidden_size** niteliği hidden_states vektörünün boyutunu tanımlar ve **num_hidden_layers** Transformer modelinin sahip olduğu katman sayısını tanımlar.

## Different loading methods

Bir modeli varsayılan yapılandırmadan yeniden başlatmak, onu rastgele değerlerle başlatır:

In [3]:
config = BertConfig()
model = BertModel(config)

# Model is randomly initialized!

Model bu durumda kullanılabilir, ancak anlamsız çıktılar verecektir; önce eğitilmesi gerekir. Modeli elimizdeki görev üzerinde sıfırdan eğitebiliriz, ancak Bölüm 1'de gördüğünüz gibi, bu uzun zaman ve çok fazla veri gerektirir ve ihmal edilemez bir çevresel etkiye sahip olur. Gereksiz ve yinelenen çabalardan kaçınmak için, daha önce eğitilmiş modelleri paylaşabilmek ve yeniden kullanabilmek zorunludur.

Önceden eğitilmiş bir Transformer modelini yüklemek basittir - bunu from_pretrained() yöntemini kullanarak yapabiliriz:

In [4]:
model = BertModel.from_pretrained("bert-base-cased")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Daha önce gördüğünüz gibi, **BertModel**'i eşdeğer **AutoModel** sınıfı ile değiştirebiliriz. Şu andan itibaren bunu yapacağız çünkü bu, kontrol noktasından bağımsız kod üretir; kodunuz bir kontrol noktası için çalışıyorsa, diğeriyle sorunsuz bir şekilde çalışmalıdır. Bu, kontrol noktası benzer bir görev için eğitildiği sürece (örneğin, bir duygu analizi görevi) mimari farklı olsa bile geçerlidir.

Yukarıdaki kod örneğinde **BertConfig** kullanmadık ve bunun yerine **bert-base-cased** tanımlayıcısı aracılığıyla önceden eğitilmiş bir model yükledik. Bu, BERT'in yazarları tarafından eğitilmiş bir model kontrol noktasıdır; model kartında bununla ilgili daha fazla ayrıntı bulabilirsiniz.

Bu model şimdi kontrol noktasının tüm ağırlıkları ile başlatılmıştır. Eğitildiği görevler üzerinde çıkarım yapmak için doğrudan kullanılabilir ve ayrıca yeni bir görev üzerinde ince ayar yapılabilir. Sıfırdan eğitmek yerine önceden eğitilmiş ağırlıklarla eğiterek hızlı bir şekilde iyi sonuçlar elde edebiliriz.

Ağırlıklar indirildi ve varsayılan olarak *~/.cache/huggingface/transformers* olan önbellek klasöründe önbelleğe alındı (böylece **from_pretrained()** yöntemine gelecekteki çağrılar onları yeniden indirmeyecek). **HF_HOME** ortam değişkenini ayarlayarak önbellek klasörünüzü özelleştirebilirsiniz.

Modeli yüklemek için kullanılan tanımlayıcı, BERT mimarisiyle uyumlu olduğu sürece Model Hub'ındaki herhangi bir modelin tanımlayıcısı olabilir. Mevcut BERT kontrol noktalarının tüm listesine buradan ulaşabilirsiniz.

## Saving methods

Bir modeli kaydetmek, bir modeli yüklemek kadar kolaydır - **from_pretrained()** yöntemine benzer olan **save_pretrained()** yöntemini kullanırız:

In [5]:
model.save_pretrained("./")

config.json dosyasına bakarsanız, model mimarisini oluşturmak için gerekli öznitelikleri tanıyacaksınız. Bu dosya ayrıca, kontrol noktasının nereden kaynaklandığı ve kontrol noktasını en son kaydettiğinizde hangi Transformers sürümünü kullandığınız gibi bazı meta verileri de içerir.

pytorch_model.bin dosyası durum sözlüğü olarak bilinir; modelinizin tüm ağırlıklarını içerir. Bu iki dosya el ele gider; yapılandırma modelinizin mimarisini bilmek için gereklidir, model ağırlıkları ise modelinizin parametreleridir.

## Using a Transformer model for inference

Artık bir modeli nasıl yükleyeceğinizi ve kaydedeceğinizi bildiğinize göre, bazı tahminler yapmak için onu kullanmayı deneyelim. Transformatör modelleri yalnızca sayıları işleyebilir - tokenizer'ın ürettiği sayılar. Ancak belirteçleri tartışmadan önce, modelin hangi girdileri kabul ettiğini inceleyelim.

Belirteçler, girdileri uygun çerçevenin tensörlerine dönüştürme işini halledebilir, ancak neler olup bittiğini anlamanıza yardımcı olmak için girdileri modele göndermeden önce yapılması gerekenlere hızlıca bir göz atacağız.

Diyelim ki birkaç dizimiz var:

In [6]:
sequences = ["Hello!", "Cool.", "Nice!"]

Tokenizer bunları tipik olarak girdi ID'leri olarak adlandırılan kelime indekslerine dönüştürür. Her dizi artık bir sayı listesidir! Elde edilen çıktı şöyledir:

In [7]:
encoded_sequences = [
    [101, 7592, 999, 102],
    [101, 4658, 1012, 102],
    [101, 3835, 999, 102],
]

Bu, kodlanmış dizilerin bir listesidir: listelerin bir listesi. Tensörler sadece dikdörtgen şekilleri kabul eder (matrisleri düşünün). Bu "dizi" zaten dikdörtgen şeklindedir, bu nedenle onu bir tensöre dönüştürmek kolaydır:

In [8]:
import torch

model_inputs = torch.tensor(encoded_sequences)

## Using the tensors as inputs to the model

Tensörleri modelle birlikte kullanmak son derece basittir - modeli sadece girdilerle çağırırız:

In [10]:
output = model(model_inputs)
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 4.4496e-01,  4.8276e-01,  2.7797e-01,  ..., -5.4032e-02,
           3.9393e-01, -9.4770e-02],
         [ 2.4943e-01, -4.4093e-01,  8.1772e-01,  ..., -3.1917e-01,
           2.2992e-01, -4.1172e-02],
         [ 1.3668e-01,  2.2518e-01,  1.4502e-01,  ..., -4.6915e-02,
           2.8224e-01,  7.5566e-02],
         [ 1.1789e+00,  1.6738e-01, -1.8187e-01,  ...,  2.4671e-01,
           1.0441e+00, -6.1968e-03]],

        [[ 3.6436e-01,  3.2465e-02,  2.0258e-01,  ...,  6.0111e-02,
           3.2451e-01, -2.0996e-02],
         [ 7.1866e-01, -4.8725e-01,  5.1740e-01,  ..., -4.4012e-01,
           1.4553e-01, -3.7545e-02],
         [ 3.3223e-01, -2.3271e-01,  9.4876e-02,  ..., -2.5268e-01,
           3.2172e-01,  8.1110e-04],
         [ 1.2523e+00,  3.5754e-01, -5.1320e-02,  ..., -3.7840e-01,
           1.0526e+00, -5.6255e-01]],

        [[ 2.4042e-01,  1.4718e-01,  1.2110e-01,  ...,  7.6062e-02,
           3.3564e-01,  2

Model birçok farklı argüman kabul etse de, yalnızca girdi kimlikleri gereklidir. Diğer argümanların ne işe yaradığını ve ne zaman gerekli olduklarını daha sonra açıklayacağız, ancak önce bir Transformer modelinin anlayabileceği girdileri oluşturan tokenizer'lara daha yakından bakmamız gerekiyor.